# Lab 5: Ensemble Models for Wine Quality Prediction
**Author:** Kate Huntsman
**Date:** April 8th, 2025  
**Objective:** 

## Introduction: 
In this lab, we apply ensemble machine learning models to predict the quality of red wine based on physicochemical characteristics. Ensemble models often outperform individual models by reducing overfitting and improving generalization. We'll evaluate various ensemble strategies and compare them using metrics like accuracy, F1 score, and the gap between training and test performance.

## Imports
In the code cell below, import the necessary Python libraries for this notebook.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    BaggingClassifier,
    VotingClassifier,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

## Section 1: Load and Inspect the Data

In [2]:
# Load the dataset (download from UCI and save in the same folder)
df = pd.read_csv("winequality-red.csv", sep=";")

# Display structure and first few rows
df.info()
df.head()

# The dataset includes 11 physicochemical input variables (features):
# ---------------------------------------------------------------
# - fixed acidity          mostly tartaric acid
# - volatile acidity       mostly acetic acid (vinegar)
# - citric acid            can add freshness and flavor
# - residual sugar         remaining sugar after fermentation
# - chlorides              salt content
# - free sulfur dioxide    protects wine from microbes
# - total sulfur dioxide   sum of free and bound forms
# - density                related to sugar content
# - pH                     acidity level (lower = more acidic)
# - sulphates              antioxidant and microbial stabilizer
# - alcohol                % alcohol by volume

# The target variable is:
# - quality (integer score from 0 to 10, rated by wine tasters)

# We will simplify this target into three categories:
#   - low (3–4), medium (5–6), high (7–8) to make classification feasible.
#   - we will also make this numeric (we want both for clarity)
# The dataset contains 1599 samples and 12 columns (11 features + target).

# Load spiral dataset
spiral = pd.read_csv("spiral.csv")

# Display basic information
spiral.info()

# Display first few rows
print(spiral.head())

FileNotFoundError: [Errno 2] No such file or directory: 'winequality-red.csv'

## Section 2: Prepare the Data

In [ ]:
# Transform the quality score into categorical labels and numeric targets
# 'low', 'medium', 'high' are easier for classification and interpretation
def quality_to_label(q):
    if q <= 4:
        return "low"
    elif q <= 6:
        return "medium"
    else:
        return "high"

def quality_to_number(q):
    if q <= 4:
        return 0
    elif q <= 6:
        return 1
    else:
        return 2

df["quality_label"] = df["quality"].apply(quality_to_label)
df["quality_numeric"] = df["quality"].apply(quality_to_number)

## Section 3: Feature Selection

In [ ]:
# Select features and the target variable
# We drop columns not needed for training (original quality columns and labels)
X = df.drop(columns=["quality", "quality_label", "quality_numeric"])
y = df["quality_numeric"]

## Section 4: Split the Data

In [ ]:
# Split the dataset into training and testing sets
# Stratify to ensure balanced class distribution in train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Section 5: Model Evaluation

In [ ]:
# Define a helper function to evaluate and compare models
# It prints performance metrics and appends them to a results list for later comparison
results = []

def evaluate_model(name, model, X_train, y_train, X_test, y_test, results):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)
    train_f1 = f1_score(y_train, y_train_pred, average="weighted")
    test_f1 = f1_score(y_test, y_test_pred, average="weighted")

    print(f"\n{name} Results")
    print("Confusion Matrix (Test):")
    print(confusion_matrix(y_test, y_test_pred))
    print(f"Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")
    print(f"Train F1 Score: {train_f1:.4f}, Test F1 Score: {test_f1:.4f}")

    results.append(
        {
            "Model": name,
            "Train Accuracy": train_acc,
            "Test Accuracy": test_acc,
            "Train F1": train_f1,
            "Test F1": test_f1,
            "Accuracy Gap": train_acc - test_acc,
            "F1 Gap": train_f1 - test_f1
        }
    )

### Run Two Models of Your Choice (RandomForest and Gradient Boosting)

In [ ]:
# Evaluate Random Forest Classifier with 100 trees
evaluate_model(
    "Random Forest (100)",
    RandomForestClassifier(n_estimators=100, random_state=42),
    X_train,
    y_train,
    X_test,
    y_test,
    results,
)

# Evaluate Gradient Boosting Classifier with 100 estimators and learning rate of 0.1
evaluate_model(
    "Gradient Boosting (100)",
    GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    X_train,
    y_train,
    X_test,
    y_test,
    results,
)

## Section 6: Compare Results

In [ ]:
# Convert the list of results into a DataFrame for easier sorting and visualization
results_df = pd.DataFrame(results)
results_df_sorted = results_df.sort_values(by="Test Accuracy", ascending=False)
display(results_df_sorted)

## Section 7: Conclusions and Insights

### Which model performed better overall?
Based on the evaluation metrics, Gradient Boosting (100) slightly outperformed Random Forest (100) in terms of both Test Accuracy and Test F1 Score. While both models are strong ensemble learners, Gradient Boosting’s sequential learning approach helped it better correct earlier errors and fine-tune performance.

### Which had the smallest accuracy/F1 gap (better generalization)?
Random Forest (100) had a smaller gap between Train and Test Accuracy and F1 Score, suggesting that it generalizes slightly better to unseen data. Gradient Boosting had higher train scores, but a slightly larger gap — a potential sign that it’s starting to overfit.

### Any signs of overfitting?
Yes — Gradient Boosting (100) showed signs of overfitting due to its high training scores and a larger performance drop on the test set. In contrast, Random Forest performed strongly with more stability between train and test sets, implying better generalization.

### Next steps or hyperparameters to tune
For Gradient Boosting, consider tuning:
- learning_rate: lower it (e.g., 0.05 or 0.01) for smoother learning.
- n_estimators: increase the number of trees.
- max_depth: reduce tree depth to prevent overfitting.

For Random Forest, you could:
- Tune max_depth to reduce complexity.
- Try max_features to limit features considered per split.
- Use cross-validation for more robust model selection.